In [ ]:
# variaveis para atualizar
trimestre = 'Pandemic';
descricao = "Infodemics and election"
nomeimag = 'Infod_election'

In [ ]:
# fazendo o dendrograma - planilha mais atual 15/11/2021 - por trimestre

from scipy.spatial.distance import pdist, squareform
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#cluster com dendrograma
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_excel("databases/electionDatabase.xlsx")

In [ ]:
listaEstado = ['AC','AL','AM','AP','BA','CE','DF','ES','GO','MA','MG','MS','MT','PA','PB','PE','PI','PR','RJ','RN','RO','RR','RS','SC','SE','SP','TO']
dadosT = df

# choosing the variables
columns = ['UF','TXInfodemicos', 'Ganhou','dtm']

dadosT = pd.DataFrame(dadosT, columns = columns)

In [ ]:
estados = dadosT.iloc[:,0].to_list()
movements = dadosT.drop(dadosT.columns[0], axis=1)

In [ ]:
# 1) problema/dados
# 2) tratamento dados, padronização
# 3) coeficiente de semelhança - medida de dissimilaridade (euclidiana)
# 4) processo de aglomeração (ward, average) - dendrograma
#    E k-means (gráfico do cotovelo) para estimar k grupos
#    CCC - Coef. Corr. Cofenética (maior que 0,7)
# 5) análises

In [ ]:
# 2) tratamento dados, padronização

scaler = StandardScaler()
movements = scaler.fit_transform(movements)


In [ ]:
# 3) coeficiente de semelhança - medida de dissimilaridade (euclidiana)

mat = pdist(movements, 'euclid')
matDistancias = squareform(mat)


In [ ]:
matDistanciastabela = pd.DataFrame(matDistancias,columns=listaEstado, index=listaEstado)

In [ ]:
# determinar a correlação cofenética
mergings = linkage(movements, method='average', metric='euclidean')

from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

c, coph_dists = cophenet(mergings, mat)
c

In [ ]:
# dataframe com as informações do LINKAGE - vai ser salvo em um arquivo excel
dfmer = pd.DataFrame(mergings)

In [ ]:
# set the fig size
fig= plt.figure(figsize=(9,5))

# Plot the dendrogram
dendrogram(mergings,
           labels=estados,
           leaf_rotation=75,
           leaf_font_size=12
          )
#plt.title(f'Dendrograma {trimestre}  {descricao}')
plt.xlabel('States')
plt.ylabel('Euclidean Distance')

# salvar cada dendrograma
plt.savefig(f'databases/{trimestre}_{nomeimag}.png')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
wcss = []
 
for i in range(2, 15):
    kmeans = KMeans(n_clusters = i)
    kmeans.fit(mergings)
    #print i, kmeans.inertia_
    wcss.append(kmeans.inertia_)
    

valorwcss = wcss    
plt.plot(range(2, 15), wcss)
plt.title(f'Metodo Elbow - Curva do Cotovelo - {trimestre}')
plt.xlabel('Numero de Clusters')
plt.ylabel('WSS') #within cluster sum of squares

plt.savefig(f'databases/{trimestre}_{nomeimag}_Elbow.png')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from math import sqrt

In [ ]:
teste = []

def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    
    teste = distances
    
    return distances.index(max(distances)) + 2

In [ ]:
# calculando a quantidade ótima de clusters
n = optimal_number_of_clusters(wcss)

In [ ]:
# criterio de numero de clusters
rotulos_k = fcluster(mergings, t = n, criterion='maxclust')

In [ ]:
from scipy.cluster.hierarchy import fcluster

# Escolhendo o ponto de corte da nossa árvore do dendrograma
# Faça uma alteração nesse valor mudam os clusters
#labels = rotulos_dist 
labels = rotulos_k

# criando um dataframe com as labes e as empresas= df
dfCluster = pd.DataFrame({'Clusters': labels, 'Estados': estados})

# Criando uma crosstab: ct
ct = pd.crosstab(dfCluster['Clusters'], dfCluster['Estados'])

# transformando a crosstab em um cluster map
sns.clustermap(ct, figsize=(8, 4), cbar_pos = None,  dendrogram_ratio=(0.001))

plt.title(f'Clustermap - {trimestre}')
plt.savefig(f'databases/{trimestre}_{nomeimag}_Map.png')

In [ ]:
dfmer = dfmer.append(pd.DataFrame(dfCluster))                          
dfmer = dfmer.append(pd.DataFrame(matDistanciastabela))

In [ ]:
dfmer.to_excel(f'databases/{trimestre}_{nomeimag}.xlsx', encoding='utf-8', index=False)